## BLSH = Buy Low Sell High

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Dependencies
import gym
import gym_anytrading

from gym_anytrading.envs import CryptoEnvLogBLSH

In [3]:
# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C

In [4]:
# Processing libraries
import numpy as np
import pandas as pd

In [5]:
# Plot libs
from matplotlib import pyplot as plt
import plotly.express as px

In [6]:
import quantstats as qs

In [7]:
#loading our dataset
df = pd.read_csv('../../data/50_log/binance_BTCUSDT_1d_from_2017_01_01_to_2022_12_31_candlesticks_signals_processed_technical_indicators_and_crypto_index_log.csv')
#viewing first 5 columns
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,SAR,ratio_high_open,ratio_low_open,ratio_close_open,time_to_chart_return,crypto_index,log_open,log_high,log_low,log_close
0,1513296000000,16334.98,17991.00,16298.45,17539.83,9181.273947,1513382399999,1.598097e+08,85478,4712.578400,...,12668.038416,1.101379,0.997764,1.073759,0.073759,1073.758890,9.701064,9.797627,9.698825,9.772230
1,1513382400000,17516.81,19539.00,17190.01,19102.66,4202.628709,1513468799999,7.629372e+07,51777,2308.847163,...,12880.956879,1.115443,0.981344,1.090533,0.089102,1169.432714,9.770916,9.880168,9.752084,9.857583
2,1513468800000,19120.19,19798.68,18510.00,18860.02,9177.183434,1513555199999,1.758674e+08,84581,4197.960835,...,13280.439467,1.035486,0.968087,0.986393,-0.012702,1154.578702,9.858500,9.893371,9.826066,9.844800
3,1513555200000,18860.04,19300.00,17029.98,18856.25,10624.633071,1513641599999,1.963392e+08,91956,4690.302146,...,13801.898709,1.023328,0.902966,0.999799,-0.000200,1154.347909,9.844801,9.867860,9.742731,9.844600
4,1513641600000,18856.25,18950.00,16300.00,17295.20,13210.748220,1513727999999,2.327906e+08,116569,5740.506184,...,14281.641213,1.004972,0.864435,0.917213,-0.082787,1058.783053,9.844600,9.849559,9.698920,9.758184


In [8]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body', 'CDLINVERTEDHAMMER', 'CDLHAMMER',
       'CDLPIERCING', 'CDLMORNINGSTAR', 'CDLSHOOTINGSTAR', 'CDLHANGINGMAN',
       'CDLDARKCLOUDCOVER', 'CDLEVENINGSTAR', 'CDLENGULFING',
       'CDLENGULFINGBULLISH', 'CDLENGULFINGBEARISH', 'CDLINVERTEDHAMMER_NEW',
       'CDLHAMMER_NEW', 'CDLPIERCING_NEW', 'CDLMORNINGSTAR_NEW',
       'CDLENGULFINGBULLISH_NEW', 'CDLSHOOTINGSTAR_NEW', 'CDLHANGINGMAN_NEW',
       'CDLDARKCLOUDCOVER_NEW', 'CDLEVENINGSTAR_NEW',
       'CDLENGULFINGBEARISH_NEW', 'MA5', 'MA10', 'MA20', 'MA60', 'MA120',
       'volume_MA5', 'volume_MA10', 'volume_MA20', 'SMA5', 'SMA10', 'SMA20',
       'SMA60', 'SMA120', 'volume_SMA5', 'volume_SMA10', 'volume_SMA20',
       'WMA5', 'W

In [9]:
#converting Date Column to DateTime Type
df['Date'] = pd.to_datetime(df['formatted_open_time'])
df.dtypes

open_time             int64
open                float64
high                float64
low                 float64
close               float64
                  ...      
log_open            float64
log_high            float64
log_low             float64
log_close           float64
Date         datetime64[ns]
Length: 95, dtype: object

In [10]:
#setting the column as index
df.set_index('Date', inplace=True)
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,SAR,ratio_high_open,ratio_low_open,ratio_close_open,time_to_chart_return,crypto_index,log_open,log_high,log_low,log_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-15,1513296000000,16334.98,17991.00,16298.45,17539.83,9181.273947,1513382399999,1.598097e+08,85478,4712.578400,...,12668.038416,1.101379,0.997764,1.073759,0.073759,1073.758890,9.701064,9.797627,9.698825,9.772230
2017-12-16,1513382400000,17516.81,19539.00,17190.01,19102.66,4202.628709,1513468799999,7.629372e+07,51777,2308.847163,...,12880.956879,1.115443,0.981344,1.090533,0.089102,1169.432714,9.770916,9.880168,9.752084,9.857583
2017-12-17,1513468800000,19120.19,19798.68,18510.00,18860.02,9177.183434,1513555199999,1.758674e+08,84581,4197.960835,...,13280.439467,1.035486,0.968087,0.986393,-0.012702,1154.578702,9.858500,9.893371,9.826066,9.844800
2017-12-18,1513555200000,18860.04,19300.00,17029.98,18856.25,10624.633071,1513641599999,1.963392e+08,91956,4690.302146,...,13801.898709,1.023328,0.902966,0.999799,-0.000200,1154.347909,9.844801,9.867860,9.742731,9.844600
2017-12-19,1513641600000,18856.25,18950.00,16300.00,17295.20,13210.748220,1513727999999,2.327906e+08,116569,5740.506184,...,14281.641213,1.004972,0.864435,0.917213,-0.082787,1058.783053,9.844600,9.849559,9.698920,9.758184


In [11]:
#df_selected_columns = df[['close', 'volume', 'CCI_14', 'ADX_14', 'ADXR_14', 'MACD_12_26_9', 'RSI_14']]
df_selected_columns = df

In [12]:
df_selected_columns

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,SAR,ratio_high_open,ratio_low_open,ratio_close_open,time_to_chart_return,crypto_index,log_open,log_high,log_low,log_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-15,1513296000000,16334.98,17991.00,16298.45,17539.83,9181.273947,1513382399999,1.598097e+08,85478,4712.578400,...,12668.038416,1.101379,0.997764,1.073759,0.073759,1073.758890,9.701064,9.797627,9.698825,9.772230
2017-12-16,1513382400000,17516.81,19539.00,17190.01,19102.66,4202.628709,1513468799999,7.629372e+07,51777,2308.847163,...,12880.956879,1.115443,0.981344,1.090533,0.089102,1169.432714,9.770916,9.880168,9.752084,9.857583
2017-12-17,1513468800000,19120.19,19798.68,18510.00,18860.02,9177.183434,1513555199999,1.758674e+08,84581,4197.960835,...,13280.439467,1.035486,0.968087,0.986393,-0.012702,1154.578702,9.858500,9.893371,9.826066,9.844800
2017-12-18,1513555200000,18860.04,19300.00,17029.98,18856.25,10624.633071,1513641599999,1.963392e+08,91956,4690.302146,...,13801.898709,1.023328,0.902966,0.999799,-0.000200,1154.347909,9.844801,9.867860,9.742731,9.844600
2017-12-19,1513641600000,18856.25,18950.00,16300.00,17295.20,13210.748220,1513727999999,2.327906e+08,116569,5740.506184,...,14281.641213,1.004972,0.864435,0.917213,-0.082787,1058.783053,9.844600,9.849559,9.698920,9.758184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,1672099200000,16919.39,16972.83,16592.37,16706.36,173749.586160,1672185599999,2.918380e+09,5047628,86096.585860,...,18069.828997,1.003159,0.980672,0.987409,-0.012591,1022.735259,9.736216,9.739369,9.716698,9.723545
2022-12-28,1672185600000,16706.06,16785.19,16465.33,16547.31,193037.565770,1672271999999,3.210894e+09,5465468,96118.187170,...,18033.558417,1.004737,0.985590,0.990497,-0.009520,1012.998485,9.723527,9.728252,9.709012,9.713979
2022-12-29,1672272000000,16547.32,16664.41,16488.91,16633.47,160998.471580,1672358399999,2.671666e+09,4553877,80977.624430,...,17998.013249,1.007076,0.996470,1.005206,0.005207,1018.273056,9.713979,9.721031,9.710443,9.719172


In [13]:
window_size = 5
start_date_id = int(df_selected_columns.index.get_loc('2018-01-06 00:00:00'))
mid_date_id = int(df_selected_columns.index.get_loc('2022-09-30 00:00:00'))
end_date_id = int(df_selected_columns.index.get_loc('2022-12-31 00:00:00'))

In [14]:
print(f'start_date_id:{start_date_id}')
print(f'mid_date_id:{mid_date_id}')
print(f'end_date_id:{end_date_id}')

start_date_id:22
mid_date_id:1746
end_date_id:1838


In [15]:
print(f'mid_date_id - start_date_id: {mid_date_id - start_date_id}')
print(f'end_date_id - mid_date_id: {end_date_id - mid_date_id}')
print(f'end_date_id - start_date_id: {end_date_id - start_date_id}')

mid_date_id - start_date_id: 1724
end_date_id - mid_date_id: 92
end_date_id - start_date_id: 1816


In [16]:
df_selected_columns.shape

(1839, 94)

In [17]:
# check prices
#df_selected_columns.loc[:, 'close'].to_numpy()[start_date_id-window_size:mid_date_id]

In [18]:
#df_selected_columns.loc[:, 'close'].to_numpy()[start_date_id - window_size + 6]

In [19]:
def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'close'].to_numpy()[start:end]
    dates = env.df.index.to_numpy()[start:end]
    signal_features = env.df.loc[:, ['RSI_14']].to_numpy()[start:end]
    return prices, signal_features, dates


class MyEnv(CryptoEnvLogBLSH):
    _process_data = my_process_data

In [20]:
#passing the data and creating our environment
env = MyEnv(df=df_selected_columns, frame_bound=(start_date_id,mid_date_id), window_size=window_size)

In [21]:
#running the test environment
state = env.reset()
while True: 
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    if done: 
        print("info", info)
        break
        
# plt.figure(figsize=(15,6))
# plt.cla()
fig = env.render_all()
fig.show()
fig.write_html("env-sample-run.html")

info {'total_reward_cash': -634.0872281154669, 'total_profit_percentage': 0.3659127718845332, 'fmt_total_profit_percentage': -63.40872281154668, 'num_of_trades': 535.0}


In [22]:
#setting up our environment for training 
env_maker = lambda: MyEnv(df=df_selected_columns, frame_bound=(start_date_id,mid_date_id), window_size=window_size)
env = DummyVecEnv([env_maker])

#Applying the Trading RL Algorithm
model = A2C('MlpPolicy', env, verbose=1) 
 
#setting the learning timesteps
#model.learn(total_timesteps=1000)
model.learn(total_timesteps=(mid_date_id - start_date_id))

Using cpu device
------------------------------------
| time/                 |          |
|    fps                | 2519     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1       |
|    explained_variance | 8.76e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 180      |
|    value_loss         | 1.81e+04 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 2358     |
|    iterations         | 200      |
|    time_elapsed       | 0        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | 0.0111   |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | -111     |
|    value_loss      

In [28]:
#Setting up the Agent Environment
env = MyEnv(df=df_selected_columns, frame_bound=(mid_date_id,end_date_id), window_size=window_size, log=True)
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

info {'total_reward_cash': -116.73889262475876, 'total_profit_percentage': 0.8832611073752418, 'fmt_total_profit_percentage': -11.673889262475823, 'num_of_trades': 29.0}


In [29]:
#Plotting our Model for Trained Trades
# mpl_fig = plt.figure(figsize=(15,6))
# plt.cla()
fig = env.render_all()
fig.show()

In [25]:
#plotly_fig = tls.mpl_to_plotly(mpl_fig)
#plotly.offline.plot(plotly_fig, filename="dmtbuoaga_1d_bcl_BLSH.html", auto_open=False)

In [26]:
#iplot(plotly_fig)

In [27]:
qs.extend_pandas()

start_index = mid_date_id
end_index = end_date_id

net_worth = pd.Series(env.history['total_profit_percentage'], index=df_selected_columns.index[start_index+1:end_index])
returns = net_worth.pct_change().iloc[1:]

qs.reports.full(returns)
qs.reports.html(returns, output='a2c_quantstats.html')

ValueError: Length of values (92) does not match length of index (91)